# Machine Learning

## Laboratorio 4
## Métodos de clasificación: RandomForest, AdaBoost y XGBoost

### Grupo: 
- Sebastián Gacitúa
- Patricia Calisto
- Bastián Guzmán
- Fabián Ragnarsson
- Matthias Clein



### Objetivo:
En este laboratorio deberá utilizar los métodos de clasificación, RandomForest, AdaBoost y XGBoost. Para ello, utilizará la
base de kaggle subidas a canvas en el módulo de tareas.
En este laboratorio deberá ajustar un modelo que permita predecir si las noticias entregadas son falsas o no, por el término
Fake News. Para ello, deberá investigar cómo tokenizar cada noticia (conteo de palabras) y determinar cuáles son las que
más influyen para esta clasificación.

In [1]:
import pandas as pd
import numpy as np
import re # Librería para utlizar expresiones regulares
import nltk
from nltk.corpus  import stopwords
from nltk.stem.porter import PorterStemmer # Librerías para implementar stemming
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

### Descargar las stopword desde la librería nltk

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matthiascleinespinoza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Cargar DataFrames

Se cargan los DataFrames y se rellenan los valores nulos de *train* con un string vacío.

In [3]:
train = pd.read_csv("./fake-news/train.csv").fillna("")
test = pd.read_csv("./fake-news/test.csv") 
submit = pd.read_csv("./fake-news/submit.csv")

Se añade una columna al DataFrame test con las etiquetas de los textos y se rellenan los valores nulos de *test* con un string vacío.

In [4]:
test["label"] = submit["label"]
test = test.fillna("")

Se imprimen las dimensiones de todos los DataFrames.

In [5]:
print(train.shape)
print(test.shape)
print(submit.shape)

(20800, 5)
(5200, 5)
(5200, 2)


### Información de los DataFrames

Se imprime la información de los DataFrames de *train* y *test*.

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20800 non-null  object
 2   author  20800 non-null  object
 3   text    20800 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      5200 non-null   int64 
 1   title   5200 non-null   object
 2   author  5200 non-null   object
 3   text    5200 non-null   object
 4   label   5200 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 203.2+ KB


### Texto de *train* y *test*

Los textos de *train* y *test* serán la concatenación de las variables *author* y *title* de cada DataFrame.

In [8]:
text_train = train["author"] + " " + train["title"]
text_test = test["author"] + " " + test["title"]

### Tokenización y Stemming

In [9]:
#Utilizaremos stemming para convertir las palabras a sus raices 
#removiendo sufijos y prefijos de estas.
port = PorterStemmer()

In [10]:
def stemming(content):
    #Utilizaremos la siguiente expresión regular que substituye todo 
    #caracter que no sea una letra de la "a - z" y reemplaza el elemento
    # con un espacio, eliminando símbolos especiales o números
    stemmed_content = re.sub('[^a-zA-Z]',' ',content) 

    stemmed_content = stemmed_content.lower() # Convierte todo a minúsculas 
    stemmed_content = stemmed_content.split() # Crea una lista separando cada una de las palbras
    
    #Aplicará stemming a toda palabra que no sea una una stopword
    #El resultado será guardado en una lista
    
    stemmed_content = [port.stem(word) for word in stemmed_content if not word in stopwords.words('english')] 
    
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

Aplicamos la función *stemming* a los textos de *train* y *test*.

In [11]:
text_train = text_train.apply(stemming)
text_test = text_test.apply(stemming)

### Conjuntos de *train* y *test*

Se crean los conjuntos de *train* y *test* a partir de los textos y sus etiquetas.
- X: Texto   
- y: Etiquetas

In [12]:
X_train = text_train
y_train = train["label"]

X_test = text_test
y_test = test['label']

In [13]:
X_train

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Length: 20800, dtype: object

In [14]:
y_train

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

### Vectorización utlizando Tfidfvectorizer

Referencias:
- [How to Use Tfidftransformer & Tfidfvectorizer?](https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.Y0MOnnZBw2x)
- [fake news 98% accuracy [NLTK]](https://www.kaggle.com/code/mohanvaddella/fake-news-98-accuracy-nltk)

Se instancia el TfidfVectorizer, se le entregan los datos de entrenamiento y luego se utiliza para transformar los conjuntos de *train* y *test* de texto a números.

In [15]:
tfidf_vectorizer = TfidfVectorizer(use_idf = True)
tfidf_vectorizer.fit(X_train)
X_train = tfidf_vectorizer.transform(X_train)
X_test = tfidf_vectorizer.transform(X_test)

### Variables más importantes

El método obtiene las palabras de mayor relevancia en la transformación numérica. El resultado es almacenado en un DataFrame que luego se ordena de manera descendiente, por lo que las 3 primeras filas serán las palabras de mayor importancia. 

In [16]:
first_vector_tfidfvectorizer = X_train[0] 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index = tfidf_vectorizer.get_feature_names_out(), columns = ["tfidf"]) 
df.sort_values(by = ["tfidf"], ascending = False).head(3)

,tfidf
chaffetz,0.367652
lucu,0.363596
darrel,0.359894


### Random Forest

Se instancia y entrena el clasificador Random Forest con el conjunto de entrenamiento.

In [17]:
RF_classifier = RandomForestClassifier(random_state = 0)
RF_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

Predicción de los resultados con el conjunto de prueba.

In [18]:
y_pred  = RF_classifier.predict(X_test)

Calculamos la exactitud del modelo:

In [19]:
RF_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy =",round(RF_accuracy*100,2),"%")

Accuracy = 63.75 %


In [20]:
RF_report=classification_report(y_test, y_pred)
print(RF_report)

              precision    recall  f1-score   support

           0       0.59      0.65      0.62      2339
           1       0.69      0.62      0.65      2861

    accuracy                           0.64      5200
   macro avg       0.64      0.64      0.64      5200
weighted avg       0.64      0.64      0.64      5200



### AdaBoost

Se instancia y entrena el clasificador AdaBoost con el conjunto de entrenamiento.

In [21]:
AB_classifier = AdaBoostClassifier(random_state = 0)
AB_classifier.fit(X_train, y_train)

AdaBoostClassifier(random_state=0)

Predicción de los resultados con el conjunto de prueba.

In [22]:
y_pred  = AB_classifier.predict(X_test)

Calculamos la exactitud del modelo:

In [23]:
AB_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy =",round(AB_accuracy*100,2),"%")

Accuracy = 63.96 %


In [24]:
AB_report=classification_report(y_test, y_pred)
print(AB_report)

              precision    recall  f1-score   support

           0       0.59      0.64      0.62      2339
           1       0.69      0.64      0.66      2861

    accuracy                           0.64      5200
   macro avg       0.64      0.64      0.64      5200
weighted avg       0.64      0.64      0.64      5200



### XGBoost

Se instala la librería del modelo XGBoost.

In [25]:
!pip install xgboost
import xgboost as xgb

Se instancia y entrena el clasificador XGBoost con el conjunto de entrenamiento.

In [26]:
XGB_classifier= xgb.XGBClassifier(random_state=0)
XGB_classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

Predicción de los resultados con el conjunto de prueba.


In [27]:
y_pred = XGB_classifier.predict(X_test)

Calculamos la exactitud del modelo:

In [28]:
XGB_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy =",round(XGB_accuracy*100,2),"%")

Accuracy = 63.71 %


In [29]:
XGB_report=classification_report(y_test, y_pred)
print(XGB_report)

              precision    recall  f1-score   support

           0       0.59      0.65      0.62      2339
           1       0.69      0.63      0.66      2861

    accuracy                           0.64      5200
   macro avg       0.64      0.64      0.64      5200
weighted avg       0.64      0.64      0.64      5200



### Comparación de los modelos

In [30]:
print("Random Forest Accuracy =",round(RF_accuracy*100,2),"%")
print("AdaBoost Accuracy =",round(AB_accuracy*100,2),"%")
print("XGBoost Accuracy =",round(XGB_accuracy*100,2),"%")
print("\n Random Forest Report \n")
print(RF_report)
print("\n AdaBoost Report \n")
print(AB_report)
print("\n XGBoost Report \n")
print(XGB_report)

Random Forest Accuracy = 63.75 %
AdaBoost Accuracy = 63.96 %
XGBoost Accuracy = 63.71 %

 Random Forest Report 

              precision    recall  f1-score   support

           0       0.59      0.65      0.62      2339
           1       0.69      0.62      0.65      2861

    accuracy                           0.64      5200
   macro avg       0.64      0.64      0.64      5200
weighted avg       0.64      0.64      0.64      5200


 AdaBoost Report 

              precision    recall  f1-score   support

           0       0.59      0.64      0.62      2339
           1       0.69      0.64      0.66      2861

    accuracy                           0.64      5200
   macro avg       0.64      0.64      0.64      5200
weighted avg       0.64      0.64      0.64      5200


 XGBoost Report 

              precision    recall  f1-score   support

           0       0.59      0.65      0.62      2339
           1       0.69      0.63      0.66      2861

    accuracy                  

### Conclusiones

- De los 3 modelos implementados el con mejor rendimiento (accuracy) es AdaBoost con un 63.96%. Cabe destacar que los otros dos modelos presentan resultados similares, siendo estos 63.75% para el Random Forest y 63.71% para el XGBoost.
- En una primera etapa de este laboratorio, se trabajó solamente considerando el dataset de entrenamiento al cual se le aplicó todo el procesamiento presentado en este notebook. Al obtener los accuracy para cada modelo quedamos sorprendidos por los excelentes resultados siendo estos cercanos al 99%. Sin embargo, cuando utilizamos el dataset de entrenamiento exclusivamente para entrenar a los modelos y el dataset de test para probarlos y obtener su exactitud, los resultados fueron muy distintos, llegando aproximadamente al 64%. Esto puede deberse a que las palabras presentes en el dataset de test no son consideradas y/o no tienen la misma importancia dentro del vectorizador que fue ajustado con el dataset de entrenamiento, lo que produce que el modelo no pueda clasificar correctamente las noticias. 
- Existen distintas formas de implementar la vectorización. En este trabajo se utilizó en primera instancia el CountVectorizer pero sus resultados no fueron tan buenos como el de Tfidfvectorizer, por lo que finalmente se realizó el cambio.
- Tfidfvectorizer también nos permitió obtener de manera sencilla las palabras más importantes para la clasificación.